In [57]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import plotly.express as px
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors

import warnings
warnings.filterwarnings('ignore')

In [58]:
cps1_data = pd.read_stata('https://users.nber.org/~rdehejia/data/cps_controls.dta')
cps3_data = pd.read_stata('https://users.nber.org/~rdehejia/data/cps_controls3.dta')
nswdw_data = pd.read_stata('https://users.nber.org/~rdehejia/data/nsw_dw.dta')

In [59]:
cps1_nsw_data = pd.concat([nswdw_data[nswdw_data.treat == 1], cps1_data], ignore_index=True)
cps3_nsw_data = pd.concat([nswdw_data[nswdw_data.treat == 1], cps3_data], ignore_index=True)

In [60]:
cps1_nsw_data

,data_id,treat,age,education,black,hispanic,married,nodegree,re74,re75,re78
0,Dehejia-Wahba Sample,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.000000,0.000000,9930.045898
1,Dehejia-Wahba Sample,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.000000,0.000000,3595.894043
2,Dehejia-Wahba Sample,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.000000,0.000000,24909.449219
3,Dehejia-Wahba Sample,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.000000,0.000000,7506.145996
4,Dehejia-Wahba Sample,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.000000,0.000000,289.789886
...,...,...,...,...,...,...,...,...,...,...,...
16172,CPS1,0.0,22.0,12.0,1.0,0.0,0.0,0.0,3975.352051,6801.435059,2757.437988
16173,CPS1,0.0,20.0,12.0,1.0,0.0,1.0,0.0,1445.938965,11832.240234,6895.071777
16174,CPS1,0.0,37.0,12.0,0.0,0.0,0.0,0.0,1733.951050,1559.370972,4221.865234
16175,CPS1,0.0,47.0,9.0,0.0,0.0,1.0,1.0,16914.349609,11384.660156,13671.929688


In [61]:
def get_ipw(X, y, random_state=0):
    # 傾向スコアを計算する
    ps_model = LogisticRegression(solver='lbfgs', random_state=random_state).fit(X, y)
    ps_score = ps_model.predict_proba(X)[:, 1]
    all_df = pd.DataFrame({'treatment': y, 'ps_score': ps_score})
    treatments = all_df.treatment.unique()
    if len(treatments) != 2:
        print('2群のマッチングしかできません。2群は必ず[0, 1]で表現してください。')
        raise ValueError
    # treatment == 1をgroup1, treatment == 0をgroup2とする。
    group1_df = all_df[all_df.treatment==1].copy()
    group2_df = all_df[all_df.treatment==0].copy()
    group1_df['weight'] = 1 / group1_df.ps_score
    group2_df['weight'] = 1 / (1 - group2_df.ps_score)
    weights = pd.concat([group1_df, group2_df]).sort_index()['weight'].values
    return weights

In [62]:
X = cps1_nsw_data[['age', 'education', 'black', 'hispanic', 'nodegree', 'married', 're74', 're75']]
y = cps1_nsw_data['treat']

In [63]:
X

,age,education,black,hispanic,nodegree,married,re74,re75
0,37.0,11.0,1.0,0.0,1.0,1.0,0.000000,0.000000
1,22.0,9.0,0.0,1.0,1.0,0.0,0.000000,0.000000
2,30.0,12.0,1.0,0.0,0.0,0.0,0.000000,0.000000
3,27.0,11.0,1.0,0.0,1.0,0.0,0.000000,0.000000
4,33.0,8.0,1.0,0.0,1.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...
16172,22.0,12.0,1.0,0.0,0.0,0.0,3975.352051,6801.435059
16173,20.0,12.0,1.0,0.0,0.0,1.0,1445.938965,11832.240234
16174,37.0,12.0,0.0,0.0,0.0,0.0,1733.951050,1559.370972
16175,47.0,9.0,0.0,0.0,1.0,1.0,16914.349609,11384.660156


In [64]:
y

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
16172    0.0
16173    0.0
16174    0.0
16175    0.0
16176    0.0
Name: treat, Length: 16177, dtype: float32

In [84]:
weights = get_ipw(X, y)
weights

array([ 8.76100354, 31.98535625,  5.23550636, ...,  1.00408922,
        1.0002702 ,  1.00043371])

In [66]:
def get_matched_dfs_using_propensity_score(X, y, random_state=0):
    # 傾向スコアを計算する
    ps_model = LogisticRegression(solver='lbfgs', random_state=random_state).fit(X, y)
    ps_score = ps_model.predict_proba(X)[:, 1]
    all_df = pd.DataFrame({'treatment': y, 'ps_score': ps_score})
    treatments = all_df.treatment.unique()
    if len(treatments) != 2:
        print('2群のマッチングしかできません。2群は必ず[0, 1]で表現してください。')
        raise ValueError
    # treatment == 1をgroup1, treatment == 0をgroup2とする。group1にマッチするgroup2を抽出するのでATTの推定になるはず
    group1_df = all_df[all_df.treatment==1].copy()
    group1_indices = group1_df.index
    group1_df = group1_df.reset_index(drop=True)
    group2_df = all_df[all_df.treatment==0].copy()
    group2_indices = group2_df.index
    group2_df = group2_df.reset_index(drop=True)

    # 全体の傾向スコアの標準偏差 * 0.2をしきい値とする
    threshold = all_df.ps_score.std() * 0.2

    matched_group1_dfs = []
    matched_group2_dfs = []
    _group1_df = group1_df.copy()
    _group2_df = group2_df.copy()

    while True:
        # NearestNeighborsで最近傍点1点を見つけ、マッチングする
        neigh = NearestNeighbors(n_neighbors=1)
        neigh.fit(_group1_df.ps_score.values.reshape(-1, 1))
        distances, indices = neigh.kneighbors(_group2_df.ps_score.values.reshape(-1, 1))
        # 重複点を削除する
        distance_df = pd.DataFrame({'distance': distances.reshape(-1), 'indices': indices.reshape(-1)})
        distance_df.index = _group2_df.index
        distance_df = distance_df.drop_duplicates(subset='indices')
        # しきい値を超えたレコードを削除する
        distance_df = distance_df[distance_df.distance < threshold]
        if len(distance_df) == 0:
            break
        # マッチングしたレコードを抽出、削除する
        group1_matched_indices = _group1_df.iloc[distance_df['indices']].index.tolist()
        group2_matched_indices = distance_df.index
        matched_group1_dfs.append(_group1_df.loc[group1_matched_indices])
        matched_group2_dfs.append(_group2_df.loc[group2_matched_indices])
        _group1_df = _group1_df.drop(group1_matched_indices)
        _group2_df = _group2_df.drop(group2_matched_indices)

    # マッチしたレコードを返す
    group1_df.index = group1_indices
    group2_df.index = group2_indices
    matched_df = pd.concat([
        group1_df.iloc[pd.concat(matched_group1_dfs).index],
        group2_df.iloc[pd.concat(matched_group2_dfs).index]
    ]).sort_index()
    matched_indices = matched_df.index

    return X.loc[matched_indices], y.loc[matched_indices]

In [67]:
X = cps1_nsw_data[['age', 'education', 'black', 'hispanic', 'nodegree', 'married', 're74', 're75']]
# 元のソースコードでは入れてるが、入れるとマッチング数が減って推定結果もおかしくなるのでやらない
#X['re74_2'] = X.re74 ** 2
#X['re75_2'] = X.re75 ** 2
y = cps1_nsw_data['treat']
matchX, matchy = get_matched_dfs_using_propensity_score(X, y)

In [89]:
matchX

,age,education,black,hispanic,nodegree,married,re74,re75
0,37.0,11.0,1.0,0.0,1.0,1.0,0.000000,0.000000
1,22.0,9.0,0.0,1.0,1.0,0.0,0.000000,0.000000
2,30.0,12.0,1.0,0.0,0.0,0.0,0.000000,0.000000
3,27.0,11.0,1.0,0.0,1.0,0.0,0.000000,0.000000
5,22.0,9.0,1.0,0.0,1.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...
13746,51.0,4.0,1.0,0.0,1.0,0.0,0.000000,0.000000
14159,39.0,10.0,1.0,0.0,1.0,0.0,844.443970,889.790283
14654,35.0,6.0,1.0,0.0,1.0,1.0,2300.178955,0.000000
15434,17.0,9.0,1.0,0.0,1.0,0.0,0.000000,2599.548096


In [90]:
matchX.index

Int64Index([    0,     1,     2,     3,     5,     6,     7,     8,     9,
               10,
            ...
            11506, 11530, 12656, 12966, 13445, 13746, 14159, 14654, 15434,
            16161],
           dtype='int64', length=328)

In [91]:
cps1_nsw_data.loc[matchX.index]

,data_id,treat,age,education,black,hispanic,married,nodegree,re74,re75,re78
0,Dehejia-Wahba Sample,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.000000,0.000000,9930.045898
1,Dehejia-Wahba Sample,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.000000,0.000000,3595.894043
2,Dehejia-Wahba Sample,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.000000,0.000000,24909.449219
3,Dehejia-Wahba Sample,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.000000,0.000000,7506.145996
5,Dehejia-Wahba Sample,1.0,22.0,9.0,1.0,0.0,0.0,1.0,0.000000,0.000000,4056.493896
...,...,...,...,...,...,...,...,...,...,...,...
13746,CPS1,0.0,51.0,4.0,1.0,0.0,0.0,1.0,0.000000,0.000000,0.000000
14159,CPS1,0.0,39.0,10.0,1.0,0.0,0.0,1.0,844.443970,889.790283,701.920105
14654,CPS1,0.0,35.0,6.0,1.0,0.0,1.0,1.0,2300.178955,0.000000,4425.791016
15434,CPS1,0.0,17.0,9.0,1.0,0.0,0.0,1.0,0.000000,2599.548096,6170.985840


In [92]:
# IPWで重み付け後のAbusolute Mean Difference
# 重みのぶんレコードを増やして計算する（もっといいやり方を知りたい）
after_weighted_df = cps1_nsw_data.loc[matchX.index][['treat', 'age', 'education', 'black', 'hispanic', 'nodegree', 'married', 're74', 're75']]
after_weighted_df

,treat,age,education,black,hispanic,nodegree,married,re74,re75
0,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.000000,0.000000
1,1.0,22.0,9.0,0.0,1.0,1.0,0.0,0.000000,0.000000
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.000000,0.000000
3,1.0,27.0,11.0,1.0,0.0,1.0,0.0,0.000000,0.000000
5,1.0,22.0,9.0,1.0,0.0,1.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...
13746,0.0,51.0,4.0,1.0,0.0,1.0,0.0,0.000000,0.000000
14159,0.0,39.0,10.0,1.0,0.0,1.0,0.0,844.443970,889.790283
14654,0.0,35.0,6.0,1.0,0.0,1.0,1.0,2300.178955,0.000000
15434,0.0,17.0,9.0,1.0,0.0,1.0,0.0,0.000000,2599.548096


In [100]:
print(after_weighted_df.values.shape)
print(after_weighted_df.values)

(328, 9)
[[1.000000e+00 3.700000e+01 1.100000e+01 ... 1.000000e+00 0.000000e+00
  0.000000e+00]
 [1.000000e+00 2.200000e+01 9.000000e+00 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 [1.000000e+00 3.000000e+01 1.200000e+01 ... 0.000000e+00 0.000000e+00
  0.000000e+00]
 ...
 [0.000000e+00 3.500000e+01 6.000000e+00 ... 1.000000e+00 2.300179e+03
  0.000000e+00]
 [0.000000e+00 1.700000e+01 9.000000e+00 ... 0.000000e+00 0.000000e+00
  2.599548e+03]
 [0.000000e+00 3.200000e+01 5.000000e+00 ... 1.000000e+00 0.000000e+00
  0.000000e+00]]


In [96]:
after_weighted_df.values[0]

array([ 1., 37., 11.,  1.,  0.,  1.,  1.,  0.,  0.], dtype=float32)

In [95]:
weights_int = (weights * 100).astype(int)
print(weights_int.size)
print(weights_int)
print(weights_int[0])
print(weights_int[1])

16177
[ 876 3198  523 ...  100  100  100]
876
3198


In [101]:
print(np.tile(after_weighted_df.values[0], (weights_int[0], 1)).shape)
print(np.tile(after_weighted_df.values[0], (weights_int[0], 1)))

(876, 9)
[[ 1. 37. 11. ...  1.  0.  0.]
 [ 1. 37. 11. ...  1.  0.  0.]
 [ 1. 37. 11. ...  1.  0.  0.]
 ...
 [ 1. 37. 11. ...  1.  0.  0.]
 [ 1. 37. 11. ...  1.  0.  0.]
 [ 1. 37. 11. ...  1.  0.  0.]]


In [110]:
weighted_df = []
for i, value in enumerate(after_weighted_df.values):
    weighted_df.append(np.tile(value, (weights_int[i], 1)))
np.array(weighted_df).shape

(328,)

In [103]:
np.concatenate(weighted_df).reshape(-1, 9).shape

(1000591, 9)

In [111]:
def calc_absolute_mean_difference(df):
    # (treatment群の平均 - control群の平均) / 全体の標準誤差
    return ((df[df.treat==1].drop('treat', axis=1).mean() - df[df.treat==0].drop('treat', axis=1).mean()) \
            / df.drop('treat', axis=1).std()).abs()

In [112]:
weighted_df = np.concatenate(weighted_df).reshape(-1, 9)
weighted_df = pd.DataFrame(weighted_df)
weighted_df.columns = after_weighted_df.columns
after_weighted_amd = calc_absolute_mean_difference(weighted_df)

In [113]:
after_weighted_amd

age          0.302460
education    0.599971
black        1.895059
hispanic     0.318400
nodegree     0.941612
married      0.519953
re74         0.550886
re75         0.414815
dtype: float32